In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [39]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.


/root/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [41]:
#from sklearn import ensemble
#from sklearn.model_selection import cross_val_score

#rfc = ensemble.RandomForestClassifier()
#X = y2015.drop('loan_status', 1)
#Y = y2015['loan_status']
#X = pd.get_dummies(X)

#cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [42]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [43]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [44]:
y2015.tail()


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [46]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,...,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,...,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,...,0,0,0,0,0,1,0,0,0,0
8,66310712.0,71035433.0,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,1,0,0,0,0
9,68476807.0,73366655.0,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,1,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [47]:
#from sklearn import ensemble
#from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)


#cross_val_score(rfc, X, Y, cv=10)

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [48]:
#y2015.head()
columns_before = set(y2015.columns)
y2015.drop(['grade', 'emp_length', 'tax_liens', 'all_util', 'home_ownership', 'purpose'], 1, inplace=True)
columns_after = set(y2015.columns)
print(columns_after)

{'open_rv_24m', 'acc_now_delinq', 'mo_sin_rcnt_tl', 'pct_tl_nvr_dlq', 'chargeoff_within_12_mths', 'last_pymnt_amnt', 'title', 'num_tl_op_past_12m', 'collections_12_mths_ex_med', 'num_rev_tl_bal_gt_0', 'total_bal_ex_mort', 'num_op_rev_tl', 'collection_recovery_fee', 'last_pymnt_d', 'mo_sin_old_il_acct', 'loan_amnt', 'pub_rec', 'mths_since_last_record', 'open_il_6m', 'mort_acc', 'total_pymnt', 'num_tl_90g_dpd_24m', 'verification_status_joint', 'mo_sin_old_rev_tl_op', 'initial_list_status', 'avg_cur_bal', 'dti', 'loan_status', 'revol_bal', 'application_type', 'mths_since_recent_bc', 'total_bal_il', 'tot_cur_bal', 'tot_hi_cred_lim', 'il_util', 'pymnt_plan', 'max_bal_bc', 'num_il_tl', 'num_bc_sats', 'total_rec_late_fee', 'inq_last_12m', 'pub_rec_bankruptcies', 'delinq_amnt', 'installment', 'percent_bc_gt_75', 'total_bc_limit', 'num_tl_120dpd_2m', 'annual_inc', 'total_acc', 'total_rec_int', 'funded_amnt_inv', 'total_rev_hi_lim', 'acc_open_past_24mths', 'total_pymnt_inv', 'annual_inc_joint', 

In [49]:
y2015[columns_after]

,open_rv_24m,acc_now_delinq,mo_sin_rcnt_tl,pct_tl_nvr_dlq,chargeoff_within_12_mths,last_pymnt_amnt,title,num_tl_op_past_12m,collections_12_mths_ex_med,num_rev_tl_bal_gt_0,...,policy_code,bc_open_to_buy,mths_since_last_delinq,mths_since_recent_inq,term,bc_util,num_accts_ever_120_pd,num_rev_accts,dti_joint,last_credit_pull_d
0,4.0,0.0,6.0,78.9,0.0,379.39,Credit card refinancing,2.0,0.0,9.0,...,1.0,263953.0,33.0,11.0,60 months,17.6,2.0,11.0,NaN,Jan-2017
1,2.0,0.0,9.0,100.0,0.0,298.58,Credit card refinancing,2.0,0.0,3.0,...,1.0,4778.0,NaN,9.0,36 months,61.8,0.0,4.0,NaN,Jan-2017
2,1.0,0.0,13.0,100.0,0.0,20807.39,Debt consolidation,0.0,0.0,3.0,...,1.0,17538.0,NaN,0.0,36 months,54.3,0.0,9.0,NaN,Jan-2017
3,1.0,0.0,19.0,91.7,0.0,858.05,Debt consolidation,0.0,0.0,9.0,...,1.0,16623.0,42.0,NaN,36 months,59.9,0.0,18.0,NaN,Jan-2017
4,16.0,0.0,0.0,100.0,0.0,8251.42,Debt consolidation,12.0,0.0,13.0,...,1.0,1375.0,NaN,0.0,36 months,45.0,0.0,17.0,NaN,Jun-2016
5,3.0,0.0,8.0,87.9,0.0,471.77,Credit card refinancing,2.0,0.0,8.0,...,1.0,15603.0,29.0,8.0,60 months,60.5,0.0,19.0,NaN,Jan-2017
6,5.0,0.0,5.0,100.0,0.0,678.49,Debt consolidation,4.0,0.0,5.0,...,1.0,18383.0,NaN,9.0,60 months,45.9,0.0,21.0,NaN,Jan-2017
7,3.0,0.0,1.0,100.0,0.0,18004.90,Debt consolidation,4.0,0.0,6.0,...,1.0,60336.0,NaN,10.0,60 months,16.7,0.0,17.0,NaN,Jan-2017
8,1.0,0.0,2.0,100.0,0.0,829.90,Debt consolidation,1.0,0.0,5.0,...,1.0,54962.0,NaN,NaN,60 months,12.1,0.0,13.0,NaN,Jan-2017
9,7.0,0.0,4.0,96.6,0.0,10128.96,Major purchase,4.0,0.0,6.0,...,1.0,4567.0,12.0,1.0,60 months,77.5,0.0,19.0,NaN,Jan-2017


In [50]:
y2015.dtypes

id                                float64
member_id                         float64
loan_amnt                         float64
funded_amnt                       float64
funded_amnt_inv                   float64
term                               object
int_rate                          float64
installment                       float64
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
title                              object
dti                               float64
delinq_2yrs                       float64
inq_last_6mths                    float64
mths_since_last_delinq            float64
mths_since_last_record            float64
open_acc                          float64
pub_rec                           float64
revol_bal                         float64
total_acc                         float64
initial_list_status               

In [51]:
cat_df=y2015.select_dtypes(include=['object'])
cat_df.head()

,term,verification_status,issue_d,loan_status,pymnt_plan,title,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint
0,60 months,Not Verified,Dec-2015,Current,n,Credit card refinancing,w,Jan-2017,Jan-2017,Jan-2017,INDIVIDUAL,NaN
1,36 months,Not Verified,Dec-2015,Current,n,Credit card refinancing,w,Jan-2017,Jan-2017,Jan-2017,INDIVIDUAL,NaN
2,36 months,Not Verified,Dec-2015,Fully Paid,n,Debt consolidation,w,Sep-2016,NaN,Jan-2017,INDIVIDUAL,NaN
3,36 months,Not Verified,Dec-2015,Current,n,Debt consolidation,w,Jan-2017,Jan-2017,Jan-2017,INDIVIDUAL,NaN
4,36 months,Verified,Dec-2015,Fully Paid,n,Debt consolidation,w,May-2016,NaN,Jun-2016,INDIVIDUAL,NaN


In [52]:
cat_df.columns.tolist()

['term',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'title',
 'initial_list_status',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'verification_status_joint']

In [53]:
cat_df['loan_status'].unique()

array(['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)',
       'In Grace Period', 'Default', 'Late (16-30 days)'], dtype=object)

In [54]:
cat_df.drop(['verification_status', 'verification_status_joint', 
             'initial_list_status', 'last_credit_pull_d', 'application_type',
             'pymnt_plan', 'issue_d', 'pymnt_plan', 'last_pymnt_d', 'next_pymnt_d'], 1, inplace=True)

In [55]:
cat_df.columns.tolist()

['term', 'loan_status', 'title']

In [56]:
cat_dummy=pd.get_dummies(cat_df)

In [57]:
cat_dummy.columns.tolist()

['term_ 36 months',
 'term_ 60 months',
 'loan_status_Charged Off',
 'loan_status_Current',
 'loan_status_Default',
 'loan_status_Fully Paid',
 'loan_status_In Grace Period',
 'loan_status_Late (16-30 days)',
 'loan_status_Late (31-120 days)',
 'title_Business',
 'title_Car financing',
 'title_Credit Card/Auto Repair',
 'title_Credit card refinancing',
 'title_Debt consolidation',
 'title_DebtC',
 'title_Green loan',
 'title_Home buying',
 'title_Home improvement',
 'title_Learning and training',
 'title_Major purchase',
 'title_Medical expenses',
 'title_Moving and relocation',
 'title_New Baby and New House (CC Consolidate)',
 'title_Other',
 'title_Pay off Lowes Card',
 'title_Paying off higher interest cards & auto',
 'title_Prescription Drug and Medical Costs',
 'title_SAVE',
 'title_Simple Loan Until Contract Is Completed',
 'title_Student Loan',
 'title_Trying to come back to reality!',
 'title_Vacation',
 'title_considerate',
 'title_new day',
 'title_new kitchen for momma!',
 

In [58]:
num_df=y2015.select_dtypes(include=['float64'])

In [59]:
num_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0.0,0.0,2.0,78.9,0.0,0.0,298100.0,31329.0,281300.0,13400.0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0.0,0.0,2.0,100.0,66.7,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0.0,0.0,0.0,100.0,20.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0.0,0.0,0.0,91.7,22.2,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0.0,0.0,12.0,100.0,50.0,1.0,38998.0,18926.0,2750.0,18248.0


In [60]:
num_df.columns.tolist()


['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'annual_inc_joint',
 'dti_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_o

In [61]:
num_df.drop(['id', 'delinq_2yrs', 
             'tot_hi_cred_lim', 'annual_inc', 'num_tl_120dpd_2m',
             'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'mths_since_recent_bc_dlq',
             'pub_rec_bankruptcies', 'inq_last_6mths'], 1, inplace=True)

In [68]:
num_df.shape

(421095, 73)

In [69]:
num_df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,dti,mths_since_last_delinq,mths_since_last_record,open_acc,...,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,percent_bc_gt_75,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,33.18,33.0,2.0,11.0,...,8.0,6.0,9.0,11.0,9.0,11.0,0.0,31329.0,281300.0,13400.0
1,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,22.44,NaN,NaN,7.0,...,3.0,5.0,4.0,4.0,3.0,7.0,66.7,55387.0,12500.0,75635.0
2,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,26.02,NaN,NaN,9.0,...,6.0,7.0,5.0,9.0,3.0,9.0,20.0,68056.0,38400.0,82117.0
3,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,21.60,42.0,NaN,16.0,...,11.0,4.0,13.0,18.0,9.0,16.0,22.2,74920.0,41500.0,42503.0
4,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,25.49,NaN,30.0,18.0,...,2.0,2.0,17.0,17.0,13.0,18.0,50.0,18926.0,2750.0,18248.0


In [76]:
num_df_dropped=num_df.dropna(axis=1, how='any', thresh=421095*0.98)

In [77]:
num_df_dropped.isna().sum()

member_id                        0
loan_amnt                        0
funded_amnt                      0
funded_amnt_inv                  0
int_rate                         0
installment                      0
dti                              0
open_acc                         0
pub_rec                          0
revol_bal                        0
total_acc                        0
out_prncp                        0
out_prncp_inv                    0
total_pymnt                      0
total_pymnt_inv                  0
total_rec_prncp                  0
total_rec_int                    0
total_rec_late_fee               0
recoveries                       0
collection_recovery_fee          0
last_pymnt_amnt                  0
collections_12_mths_ex_med       0
policy_code                      0
acc_now_delinq                   0
tot_coll_amt                     0
tot_cur_bal                      0
total_rev_hi_lim                 0
acc_open_past_24mths             0
avg_cur_bal         

In [78]:
indices = np.where(num_df_dropped.isna())

In [80]:
num_df_dropped.iloc[101]

member_id                     73306061.00
loan_amnt                         5000.00
funded_amnt                       5000.00
funded_amnt_inv                   5000.00
int_rate                             9.80
installment                        160.87
dti                                  6.52
open_acc                            11.00
pub_rec                              1.00
revol_bal                         4838.00
total_acc                           20.00
out_prncp                         3493.07
out_prncp_inv                     3493.07
total_pymnt                       1923.63
total_pymnt_inv                   1923.63
total_rec_prncp                   1506.93
total_rec_int                      416.70
total_rec_late_fee                   0.00
recoveries                           0.00
collection_recovery_fee              0.00
last_pymnt_amnt                    160.87
collections_12_mths_ex_med           0.00
policy_code                          1.00
acc_now_delinq                    

In [79]:
print(indices)

(array([   101,    101,    101, ..., 420984, 420984, 420984]), array([29, 30, 37, ..., 30, 37, 48]))


In [85]:
row_indices=np.unique(indices[0]).tolist()
print(len(row_indices))

4504


In [86]:
num_df_dropped.drop(row_indices, inplace=True)

/root/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [88]:

pca = decomposition.PCA(n_components=3)
pca.fit(num_df_dropped)
transform_df = pca.transform(num_df_dropped)

In [90]:

pca_df = pd.DataFrame(data = transform_df
             , columns = ['pc 1', 'pc 2', 'pc 3'])

In [101]:
pca_df.tail()


,pc 1,pc 2,pc 3
416586,1.849300e+07,-119273.968144,-11235.009163
416587,1.844293e+07,-11166.829871,-62879.131311
416588,1.861289e+07,-106857.025968,4071.983172
416589,1.837305e+07,-73896.369012,39829.693366
416590,1.861298e+07,-105356.308335,-10276.397823


In [108]:
#cat_dummy.drop(row_indices, inplace=True)
cat_dummy.reset_index(drop=True, inplace=True)

In [109]:
cat_dummy.index

RangeIndex(start=0, stop=416591, step=1)

In [110]:
cat_dummy.shape

(416591, 36)

In [111]:
cat_pca = pd.concat([cat_dummy,pca_df],axis=1)

In [112]:
cat_pca.tail()

,term_ 36 months,term_ 60 months,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),title_Business,...,title_Student Loan,title_Trying to come back to reality!,title_Vacation,title_considerate,title_new day,title_new kitchen for momma!,title_odymeds,pc 1,pc 2,pc 3
416586,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.849300e+07,-119273.968144,-11235.009163
416587,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.844293e+07,-11166.829871,-62879.131311
416588,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.861289e+07,-106857.025968,4071.983172
416589,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.837305e+07,-73896.369012,39829.693366
416590,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.861298e+07,-105356.308335,-10276.397823


In [113]:
target=['loan_status_Charged Off',
 'loan_status_Current',
 'loan_status_Default',
 'loan_status_Fully Paid',
 'loan_status_In Grace Period',
 'loan_status_Late (16-30 days)',
 'loan_status_Late (31-120 days)']

In [114]:
target_df=cat_pca[target]

In [119]:

cat_pca_variable=cat_pca.drop(['loan_status_Charged Off',
 'loan_status_Current',
 'loan_status_Default',
 'loan_status_Fully Paid',
 'loan_status_In Grace Period',
 'loan_status_Late (16-30 days)',
 'loan_status_Late (31-120 days)'], axis=1)

In [121]:
from sklearn.model_selection import train_test_split
cat_pca_variable_train, cat_pca_variable_test, target_df_train, target_df_test = train_test_split(
    cat_pca_variable, target_df, test_size=0.2, random_state=42)

In [124]:
#random forest classifer using 10 fold cross validation if below numerical columns
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
scores = cross_val_score(clf, cat_pca_variable, target_df, cv=5)
scores                                              

array([0.77238085, 0.73668355, 0.68869872, 0.63792938, 0.57981469])